## Importing libraries

In [1]:
import os
import sys
from pathlib import Path
import time
import numpy as np
import cv2
import torch
import torch.backends.cudnn as cudnn
import io
import base64
import datetime
import argparse
from models.common import DetectMultiBackend
from utils.augmentations import letterbox 
from utils.dataloaders import IMG_FORMATS, VID_FORMATS, LoadImages, LoadStreams

from utils.plots import Annotator, colors, save_one_box
from utils.torch_utils import select_device, time_sync
from PIL import Image
from scipy.spatial import distance as dist
import threading 

from utils.general import (LOGGER, check_file, check_img_size, check_imshow, check_requirements, colorstr,
                           increment_path, non_max_suppression, print_args, scale_boxes, strip_optimizer, xyxy2xywh)

/home/smartcity/.local/lib/python3.8/site-packages/torch/cuda/__init__.py:83: UserWarning: CUDA initialization: Unexpected error from cudaGetDeviceCount(). Did you run some cuda functions before calling NumCudaDevices() that might have already set an error? Error 804: forward compatibility was attempted on non supported HW (Triggered internally at  ../c10/cuda/CUDAFunctions.cpp:109.)
  return torch._C._cuda_getDeviceCount() > 0


In [2]:


device = select_device('cpu')#Set 0 if you have GPU
model = DetectMultiBackend('yolov5s.pt', device=device, dnn=False, data='data/coco128.yaml')

YOLOv5 🚀 2023-3-7 Python-3.8.10 torch-1.12.0+cu102 CPU

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients


In [3]:
def perform_prediction(img0,model):
    
    stride=model.stride
    img0=img0.copy() 
    img = letterbox(img0, 640, stride=stride, auto=True)[0]  # Resize image keeping aspect ratio
    cv2.imwrite("car1.jpg",img)
    height,width ,_=img.shape
    img = img.transpose((2, 0, 1))[::-1]  # HWC to CHW, BGR to RGB
    img = np.ascontiguousarray(img)
    im = torch.from_numpy(img).to(device) # converting image into torch of cpu type
    im = im.float() # uint8 to fp16/32
    im /= 255  # 0 - 255 to 0.0 - 1.0
    if len(im.shape) == 3:
        im = im[None]  # expand for batch dim
    dt = [0.0,0.0,0.0]    
    pred = model(im, augment=False, visualize=False) # Passing the image to 
    seen = 0
    pred = non_max_suppression(pred, conf_thres=0.45, iou_thres=0.45, classes=[0,1,2,3,4,6] , max_det=1000)
    det=pred[0]
    det[:, :4] = scale_boxes(im.shape[2:], det[:, :4], img0.shape).round()
    det[:, :4].cpu().numpy()
    return det[:, :4].cpu().numpy(),det[:, -1].cpu().numpy(),height,width

In [4]:
def convert_pred(predictions,classes,height,width):
    predictions.shape[0]
    x_norm, y_norm, w_norm, h_norm =[],[],[],[]
    for i in range(predictions.shape[0]):
        
        x1, y1, x2,y2 =predictions[i]
        
        x_norm.append(((x1 + x2) / (2 * width)))
        y_norm.append((y1 + y2) / (2 * height))
        w_norm.append((x2 - x1) / width)
        h_norm.append((y2 - y1) / height)
        
    predictions_anotatedformat= np.column_stack([classes,x_norm, y_norm, w_norm, h_norm ])
    return predictions_anotatedformat
    

In [11]:
import matplotlib.pyplot as plt 
im=cv2.imread("./cars.jpg") ## Reading iput images
cv2.imshow('image',im)
cv2.waitKey(0)
  
# closing all open windows
cv2.destroyAllWindows()

In [6]:
predictions,classes,height,width=perform_prediction(im,model)
converted_pred=convert_pred(predictions,classes,height,width)
np.savetxt('car1.txt',converted_pred,fmt='%.0f %.6f %.6f %.6f %.6f') 